## Create new manifestation measurements

The following code is used to create new statistics. They can be saved to a CSV file and be taken into account in the section above about existing measurements.

In [ ]:
corpus = pd.read_csv('./2022-06-20_with-duplicate-removing/csv/integrated-data-enriched.csv', index_col='targetIdentifier')
corpusNLFR = corpus[(corpus['sourceLanguage'] == 'Dutch') & (corpus['targetLanguage'] == 'French') ]
corpusFRNL = corpus[(corpus['sourceLanguage'] == 'French') & (corpus['targetLanguage'] == 'Dutch') ]
corpusOther = corpus[ ((corpus['sourceLanguage'] != 'Dutch') & (corpus['sourceLanguage'] != 'French')) | ((corpus['targetLanguage'] != 'Dutch') & (corpus['targetLanguage'] != 'French'))]

In [ ]:
comment = "New KBR data dump with plenty of additions and refinements."
measurements = pd.DataFrame([
  utils.createCorpusMeasurements(corpusFRNL, 'FR-NL', comment),
  utils.createCorpusMeasurements(corpusNLFR, 'NL-FR', comment),
  utils.createCorpusMeasurements(corpusOther, 'OTHER', comment)
])

In [ ]:
measurements

In [ ]:
#measurements.to_csv('2022-06-20-translation-stats.csv', index=False)

In [ ]:
## Create new contributor measurements

In [ ]:
personContributors = pd.read_csv('./2022-06-20_with-duplicate-removing/csv/contributors-persons.csv', index_col='contributorID')

In [ ]:
personComment = "New KBR data dump with plenty of additions and refinements."
personContributorsMeasurements = pd.DataFrame([
    utils.createContributorCorpusMeasurements(personContributors, personComment)
])

In [ ]:
personContributorsMeasurements

In [ ]:
#personContributorsMeasurements.to_csv('2022-06-20-person-contributor-stats.csv', index=False)